In [127]:
from get_data import fetch_data # Module to fetch and process data
import pandas as pd
import numpy as np

import folium
import plotly
from dateutil.relativedelta import relativedelta

In [2]:
collision_df = fetch_data()

Data successfully downloaded and processed!


In [162]:
collisions = collision_df.copy()
collisions['CRASH DATE'] = pd.to_datetime(collisions['CRASH DATE'])
collisions.drop(columns=['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5'], inplace=True)

collisions.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,COLLISION_ID,CRASH YEAR,CRASH MONTH-YEAR,CRASH HOUR
0,2021-04-14,2022-01-10 05:32:00,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,4407480,2021,2021-04,5
1,2021-04-13,2022-01-10 21:35:00,BROOKLYN,11217,40.68358,-73.97617,1.0,0.0,1,0,0,0,0,0,NaN,4407147,2021,2021-04,21
2,2021-04-15,2022-01-10 16:15:00,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Pavement Slippery,4407665,2021,2021-04,16
3,2021-04-13,2022-01-10 16:00:00,BROOKLYN,11222,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,4407811,2021,2021-04,16
4,2021-04-12,2022-01-10 08:25:00,NaN,NaN,0.00000,0.00000,0.0,0.0,0,0,0,0,0,0,NaN,4406885,2021,2021-04,8


In [230]:
last_updated = collisions['CRASH TIME'].max()

last_updated.strftime("%m/%d/%Y")

'01/10/2022'

In [163]:
grouped_by_month = collisions.groupby(['CRASH YEAR', 'CRASH MONTH-YEAR']).agg({
    'COLLISION_ID': 'count',
    'NUMBER OF PERSONS INJURED': 'sum',
    'NUMBER OF PERSONS KILLED': 'sum',
    'NUMBER OF PEDESTRIANS INJURED': 'sum',
    'NUMBER OF PEDESTRIANS KILLED': 'sum',
    'NUMBER OF CYCLIST INJURED': 'sum',
    'NUMBER OF CYCLIST KILLED': 'sum',
    'NUMBER OF MOTORIST INJURED': 'sum',
    'NUMBER OF MOTORIST KILLED': 'sum'
})
grouped_by_month.reset_index(inplace=True)

In [164]:
# grouped_by_month['cyclist_killed_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF CYCLIST KILLED'].cumsum()
# grouped_by_month

In [165]:
grouped_by_month['collisions_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['COLLISION_ID'].cumsum()

grouped_by_month['person_injured_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF PERSONS INJURED'].cumsum()
grouped_by_month['person_killed_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF PERSONS KILLED'].cumsum()

grouped_by_month['peds_injured_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF PEDESTRIANS INJURED'].cumsum()
grouped_by_month['peds_killed_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF PEDESTRIANS KILLED'].cumsum()

grouped_by_month['cyclist_injured_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF CYCLIST INJURED'].cumsum()
grouped_by_month['cyclist_killed_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF CYCLIST KILLED'].cumsum()


grouped_by_month.reset_index(inplace=True)
grouped_by_month.head()

,index,CRASH YEAR,CRASH MONTH-YEAR,COLLISION_ID,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,collisions_cumsum,person_injured_cumsum,person_killed_cumsum,peds_injured_cumsum,peds_killed_cumsum,cyclist_injured_cumsum,cyclist_killed_cumsum
0,0,2012,2012-07,16992,4855.0,22.0,855,6,450,3,3550,13,16992,4855.0,22.0,855,6,450,3
1,1,2012,2012-08,17142,4751.0,27.0,850,15,508,1,3393,11,34134,9606.0,49.0,1705,21,958,4
2,2,2012,2012-09,16535,4689.0,26.0,893,13,434,1,3362,12,50669,14295.0,75.0,2598,34,1392,5
3,3,2012,2012-10,16864,4572.0,21.0,977,13,329,1,3260,7,67533,18867.0,96.0,3575,47,1721,6
4,4,2012,2012-11,15889,3995.0,14.0,1037,8,277,0,2681,6,83422,22862.0,110.0,4612,55,1998,6


In [166]:
max_date = collisions['CRASH MONTH-YEAR'].max()

collisions_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['collisions_cumsum']

persons_injured_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['person_injured_cumsum']
persons_killed_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['person_killed_cumsum']

peds_injured_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['peds_injured_cumsum']
peds_killed_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['peds_killed_cumsum']

cyclists_injured_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['cyclist_injured_cumsum']
cyclists_killed_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['cyclist_killed_cumsum']



In [168]:
last_year = max_date.to_timestamp() - pd.DateOffset(years=1)
last_YTD= last_year.to_period('M')
last_YTD

collisions_YTD_previous = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['collisions_cumsum']

persons_injured_YTD_previous = persons_injured_YTD.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['person_injured_cumsum']
persons_killed_YTD_previous = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['person_killed_cumsum']

peds_injured_YTD_previous = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['peds_injured_cumsum']
peds_killed_YTD_previous = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['peds_killed_cumsum']

cyclists_injured_YTD_previous = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['cyclist_injured_cumsum']
cyclists_killed_YTD_previous = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['cyclist_killed_cumsum']

In [194]:
collisions_perc_change = (collisions_YTD.values - collisions_YTD_previous.values)/collisions_YTD_previous.values*100

peds_injured_perc_change = (peds_injured_YTD.values - peds_injured_YTD_previous.values)/peds_injured_YTD_previous.values*100
peds_killed_perc_change = (peds_killed_YTD.values - peds_killed_YTD_previous.values)/peds_killed_YTD_previous.values*100

cyclists_injured_perc_change = (cyclists_injured_YTD.values - cyclists_injured_YTD_previous.values)/cyclists_injured_YTD_previous.values*100
cyclists_killed_perc_change = (cyclists_killed_YTD.values - cyclists_killed_YTD_previous.values)/cyclists_killed_YTD_previous.values*100



In [198]:
grouped_by_month[grouped_by_month['CRASH YEAR'] == 2021]

,index,CRASH YEAR,CRASH MONTH-YEAR,COLLISION_ID,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,collisions_cumsum,person_injured_cumsum,person_killed_cumsum,peds_injured_cumsum,peds_killed_cumsum,cyclist_injured_cumsum,cyclist_killed_cumsum
102,102,2021,2021-01,7689,3112.0,17.0,529,9,284,2,2300,6,7689,3112.0,17.0,529,9,284,2
103,103,2021,2021-02,6962,2549.0,12.0,389,9,142,0,2018,3,14651,5661.0,29.0,918,18,426,2
104,104,2021,2021-03,8258,3526.0,18.0,530,10,324,0,2632,8,22909,9187.0,47.0,1448,28,750,2
105,105,2021,2021-04,8749,4094.0,27.0,541,15,355,0,3030,10,31658,13281.0,74.0,1989,43,1105,2
106,106,2021,2021-05,10281,5035.0,33.0,642,17,511,1,3639,13,41939,18316.0,107.0,2631,60,1616,3
107,107,2021,2021-06,10606,5085.0,27.0,627,6,547,5,3626,15,52545,23401.0,134.0,3258,66,2163,8
108,108,2021,2021-07,9998,4826.0,29.0,569,9,496,3,3507,16,62543,28227.0,163.0,3827,75,2659,11
109,109,2021,2021-08,9872,4909.0,27.0,585,12,532,0,3545,13,72415,33136.0,190.0,4412,87,3191,11
110,110,2021,2021-09,9883,4968.0,23.0,713,7,498,3,3480,13,82298,38104.0,213.0,5125,94,3689,14
111,111,2021,2021-10,10192,5027.0,33.0,805,15,543,3,3417,12,92490,43131.0,246.0,5930,109,4232,17


In [ ]:
grouped_by_month[grouped_by_month['CRASH YEAR'] == 2021]

In [44]:
########
# KPIs #
########

max_date = collisions['CRASH MONTH-YEAR'].max()

grouped_by_month_cumsum = grouped_by_month.cumsum(axis=0).reset_index()
grouped_by_month_cumsum




,CRASH MONTH-YEAR,COLLISION_ID,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED
0,2012-07,16992,4855.0,22.0,855,6,450,3,3550,13
1,2012-08,34134,9606.0,49.0,1705,21,958,4,6943,24
2,2012-09,50669,14295.0,75.0,2598,34,1392,5,10305,36
3,2012-10,67533,18867.0,96.0,3575,47,1721,6,13565,43
4,2012-11,83422,22862.0,110.0,4612,55,1998,6,16246,49
...,...,...,...,...,...,...,...,...,...,...
110,2021-09,1827084,512169.0,2398.0,94760,1240,43406,181,372204,967
111,2021-10,1837276,517196.0,2431.0,95565,1255,43949,184,375621,979
112,2021-11,1846622,521692.0,2455.0,96329,1266,44341,185,378773,991
113,2021-12,1855060,525748.0,2473.0,97121,1272,44664,186,381551,1001


In [123]:
persons_killed_YTD = grouped_by_month_cumsum.loc[grouped_by_month_cumsum['CRASH MONTH-YEAR'] == max_date]['NUMBER OF PERSONS KILLED']
persons_killed_YTD

114    2477.0
Name: NUMBER OF PERSONS KILLED, dtype: float64

In [35]:
print(f'Data through {max_date}')

Data through 2022-01


In [14]:
m = folium.Map(location=[40.7128, -74.0060])




m

In [31]:
collisions['CRASH TIME'] = pd.to_datetime(collisions['CRASH TIME'])
collisions['CRASH HOUR'] = collisions['CRASH TIME'].dt.floor('H')

In [32]:
collisions.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,...,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,CRASH YEAR,CRASH MONTH-YEAR,CRASH HOUR,HOUR
0,04/14/2021,2022-01-10 05:32:00,NaN,NaN,NaN,NaN,0.0,0.0,0,0,...,NaN,Sedan,Sedan,NaN,NaN,NaN,2021,2021-04,2022-01-10 05:00:00,6
1,04/13/2021,2022-01-10 21:35:00,BROOKLYN,11217,40.68358,-73.97617,1.0,0.0,1,0,...,NaN,Sedan,NaN,NaN,NaN,NaN,2021,2021-04,2022-01-10 21:00:00,22
2,04/15/2021,2022-01-10 16:15:00,NaN,NaN,NaN,NaN,0.0,0.0,0,0,...,NaN,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,2021,2021-04,2022-01-10 16:00:00,16
3,04/13/2021,2022-01-10 16:00:00,BROOKLYN,11222,NaN,NaN,0.0,0.0,0,0,...,NaN,Sedan,NaN,NaN,NaN,NaN,2021,2021-04,2022-01-10 16:00:00,16
4,04/12/2021,2022-01-10 08:25:00,NaN,NaN,0.00000,0.00000,0.0,0.0,0,0,...,NaN,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN,2021,2021-04,2022-01-10 08:00:00,8


In [50]:
collisions.fillna({'CONTRIBUTING FACTOR VEHICLE 1':1,'CONTRIBUTING FACTOR VEHICLE 2':1, 'CONTRIBUTING FACTOR VEHICLE 3':1, 'CONTRIBUTING FACTOR VEHICLE 4':1, 'CONTRIBUTING FACTOR VEHICLE 5':1}, inplace=True)

collisions['MULTI-CAR'] =  collisions[['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5']].sum(numeric_only=True)



In [51]:
collisions.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,MULTI-CAR
0,04/14/2021,5:32,NaN,NaN,NaN,NaN,NaN,BRONX WHITESTONE BRIDGE,NaN,NaN,...,1,1,1,4407480,Sedan,Sedan,NaN,NaN,NaN,NaN
1,04/13/2021,21:35,BROOKLYN,11217,40.68358,-73.97617,"(40.68358, -73.97617)",NaN,NaN,620 ATLANTIC AVENUE,...,1,1,1,4407147,Sedan,NaN,NaN,NaN,NaN,NaN
2,04/15/2021,16:15,NaN,NaN,NaN,NaN,NaN,HUTCHINSON RIVER PARKWAY,NaN,NaN,...,1,1,1,4407665,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN
3,04/13/2021,16:00,BROOKLYN,11222,NaN,NaN,NaN,VANDERVORT AVENUE,ANTHONY STREET,NaN,...,1,1,1,4407811,Sedan,NaN,NaN,NaN,NaN,NaN
4,04/12/2021,8:25,NaN,NaN,0.00000,0.00000,"(0.0, 0.0)",EDSON AVENUE,NaN,NaN,...,1,1,1,4406885,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN,NaN


In [52]:
collisions.drop(['LOCATION', 'COLLISION_ID', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME'], axis=1, inplace=True)
collisions['CRASH YEAR'] = pd.DatetimeIndex(collisions['CRASH DATE']).year
collisions['CRASH MONTH-YEAR'] = pd.to_datetime(collisions['CRASH DATE']).dt.to_period('M')
collisions.head()


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,...,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,MULTI-CAR,CRASH YEAR,CRASH MONTH-YEAR
0,04/14/2021,5:32,NaN,NaN,NaN,NaN,0.0,0.0,0,0,...,1,1,Sedan,Sedan,NaN,NaN,NaN,NaN,2021,2021-04
1,04/13/2021,21:35,BROOKLYN,11217,40.68358,-73.97617,1.0,0.0,1,0,...,1,1,Sedan,NaN,NaN,NaN,NaN,NaN,2021,2021-04
2,04/15/2021,16:15,NaN,NaN,NaN,NaN,0.0,0.0,0,0,...,1,1,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN,2021,2021-04
3,04/13/2021,16:00,BROOKLYN,11222,NaN,NaN,0.0,0.0,0,0,...,1,1,Sedan,NaN,NaN,NaN,NaN,NaN,2021,2021-04
4,04/12/2021,8:25,NaN,NaN,0.00000,0.00000,0.0,0.0,0,0,...,1,1,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN,NaN,2021,2021-04


In [53]:
list(collisions['VEHICLE TYPE CODE 1'].unique())

['Sedan',
 'Station Wagon/Sport Utility Vehicle',
 'Taxi',
 '�MBU',
 'Pick-up Truck',
 'Box Truck',
 'Bus',
 nan,
 'Ambulance',
 'Tow Truck / Wrecker',
 'E-Bike',
 'Van',
 'Flat Bed',
 'Moped',
 'Tractor Truck Diesel',
 'AMBULANCE',
 'Bike',
 'Trailer',
 'Motorcycle',
 'Garbage or Refuse',
 'SCHOOL BUS',
 'Lift Boom',
 'scooter',
 'CATER',
 'PKUP',
 'Carry All',
 'FDNY Ambul',
 '3-Door',
 'Beverage Truck',
 'SEMI TRAIL',
 'Dump',
 'Convertible',
 '4 dr sedan',
 'dump',
 'Flat Rack',
 'GARBAGE TR',
 'Motorbike',
 'Tanker',
 'Armored Truck',
 'MOPED',
 'SW/VAN',
 'Bulk Agriculture',
 'Motorscooter',
 'UTILITY',
 'Concrete Mixer',
 'Open Body',
 'E-Scooter',
 'unknown',
 'fire truck',
 'Glass Rack',
 'COMMERCIAL',
 'PSD',
 'Scooter',
 'MTA',
 'Multi-Wheeled Vehicle',
 'PK',
 'Minicycle',
 'TRUCK',
 'Truck',
 'Chassis Cab',
 'LIMO',
 'Tractor Truck Gasoline',
 'MACK',
 'Refrigerated Van',
 'van',
 'CARRIER',
 'Fire truck',
 'ambulance',
 'NYPD VAN',
 '2 dr sedan',
 'MTR H',
 'Pallet',
 'AM

In [54]:
collisions['VEHICLE TYPE CODE 2'] = collisions['VEHICLE TYPE CODE 1'].str.title()
collisions['VEHICLE TYPE CODE 2'].replace({'scoo': 'Scooter', '11111': np.nan}, inplace=True)
collisions['VEHICLE TYPE CODE 2'].unique()


array(['Sedan', 'Station Wagon/Sport Utility Vehicle', 'Taxi', '�Mbu',
       'Pick-Up Truck', 'Box Truck', 'Bus', nan, 'Ambulance',
       'Tow Truck / Wrecker', 'E-Bike', 'Van', 'Flat Bed', 'Moped',
       'Tractor Truck Diesel', 'Bike', 'Trailer', 'Motorcycle',
       'Garbage Or Refuse', 'School Bus', 'Lift Boom', 'Scooter', 'Cater',
       'Pkup', 'Carry All', 'Fdny Ambul', '3-Door', 'Beverage Truck',
       'Semi Trail', 'Dump', 'Convertible', '4 Dr Sedan', 'Flat Rack',
       'Garbage Tr', 'Motorbike', 'Tanker', 'Armored Truck', 'Sw/Van',
       'Bulk Agriculture', 'Motorscooter', 'Utility', 'Concrete Mixer',
       'Open Body', 'E-Scooter', 'Unknown', 'Fire Truck', 'Glass Rack',
       'Commercial', 'Psd', 'Mta', 'Multi-Wheeled Vehicle', 'Pk',
       'Minicycle', 'Truck', 'Chassis Cab', 'Limo',
       'Tractor Truck Gasoline', 'Mack', 'Refrigerated Van', 'Carrier',
       'Nypd Van', '2 Dr Sedan', 'Mtr H', 'Pallet', 'Ambulane', 'Amb',
       'Mailtruck', 'Com', 'Util', 'Pick Up

In [55]:
collisions.groupby('CRASH YEAR').sum()

,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,MULTI-CAR
CRASH YEAR,,,,,,,,,,,
2012,3.479906e+06,-6.317853e+06,27453.0,137.0,5906,72,2210,6,19331,59,0.0
2013,7.001534e+06,-1.271036e+07,55124.0,297.0,11988,176,4075,11,39060,110,0.0
2014,7.034421e+06,-1.276935e+07,51223.0,262.0,11036,133,4000,20,36176,109,0.0
2015,7.451619e+06,-1.352611e+07,51358.0,243.0,10084,133,4281,15,36992,95,0.0
2016,7.826467e+06,-1.420810e+07,60317.0,246.0,11090,149,4975,18,44010,72,0.0
2017,8.836971e+06,-1.604679e+07,60657.0,256.0,11151,127,4889,27,44616,107,0.0
2018,8.802428e+06,-1.598224e+07,61939.0,231.0,11123,123,4725,10,46068,98,0.0
2019,7.893567e+06,-1.432579e+07,61389.0,244.0,10568,131,4986,31,45834,82,0.0
2020,4.227987e+06,-7.673009e+06,44605.0,269.0,6689,102,5576,29,32340,138,0.0


In [56]:
cols_for_kepler = ['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED']

In [57]:
kepler_df = collision_df[cols_for_kepler].dropna()

In [58]:
kepler_df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED
1,04/13/2021,21:35,BROOKLYN,11217,40.683580,-73.97617,1.0,0.0,1,0,0,0,0,0
13,05/21/2019,22:50,BROOKLYN,11201,40.697540,-73.98312,0.0,0.0,0,0,0,0,0,0
15,02/26/2021,14:50,BRONX,10461,40.843464,-73.83600,0.0,0.0,0,0,0,0,0,0
17,03/31/2021,22:20,BROOKLYN,11234,40.626457,-73.91800,1.0,0.0,0,0,0,0,1,0
18,04/06/2021,22:58,STATEN ISLAND,10312,40.526894,-74.16728,7.0,0.0,0,0,0,0,7,0


In [23]:
for row in collision_df.values:
    print(row)

In [17]:
# Load a map with data and config and height
from keplergl import KeplerGl
map_2 = KeplerGl()
map_2.add_data(data=kepler_df, name='Summary of Vehile Collisions in NYC')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [ ]:
map_2